# Metapath Null Distributions

This notebook generates degree-aware null distributions for multiple metapaths using the compositional null models trained in notebook 13.

**Workflow:**
1. Load trained null models for each edge type
2. For each metapath, compute compositional null predictions
3. Compare Hetionet metapath frequencies vs null distributions
4. Statistical tests: KS test, correlation analysis
5. Detect anomalous paths in Hetionet
6. Save results and visualizations

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
from pathlib import Path
import pickle
from scipy.stats import ks_2samp, pearsonr
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Setup paths
repo_dir = Path('/Users/lucas/Library/CloudStorage/OneDrive-TheUniversityofColoradoDenver/Repositories/Context-Aware-Path-Probability')
data_dir = repo_dir / 'data'
results_dir = repo_dir / 'results' / 'metapath_null_distributions'
results_dir.mkdir(parents=True, exist_ok=True)

models_dir = repo_dir / 'results' / 'null_models'

print(f"Results will be saved to: {results_dir}")

## Load Trained Null Models

In [ ]:
def load_null_models(edge_type, model_type='rf'):
    """
    Load trained null models for an edge type.
    
    Args:
        edge_type: Edge type abbreviation (e.g., 'CbG', 'GpPW')
        model_type: 'rf' for Random Forest or 'poly_logreg' for Polynomial Logistic Regression
    
    Returns:
        dict: Loaded model and scaler
    """
    model_file = models_dir / f'{edge_type}_{model_type}_model.pkl'
    
    if not model_file.exists():
        raise FileNotFoundError(f"Model file not found: {model_file}")
    
    with open(model_file, 'rb') as f:
        model_data = pickle.load(f)
    
    return model_data

# Test loading
test_model = load_null_models('CbG', 'rf')
print(f"Successfully loaded model with keys: {test_model.keys()}")

## Load Node Degree Information

In [ ]:
def load_node_degrees(edge_type):
    """
    Load source and target node degrees for an edge type from Hetionet.
    
    Returns:
        tuple: (source_degrees, target_degrees) as numpy arrays
    """
    edge_file = data_dir / 'hetionet-v1.0' / 'hetmat' / 'edges' / f'{edge_type}.sparse.npz'
    matrix = sp.load_npz(str(edge_file))
    
    # Compute degrees
    source_degrees = np.array(matrix.sum(axis=1)).flatten()
    target_degrees = np.array(matrix.sum(axis=0)).flatten()
    
    return source_degrees, target_degrees

def get_intermediate_node_degree_distribution(node_type):
    """
    Get degree distribution for intermediate nodes of a given type.
    Uses all edges involving this node type.
    
    Args:
        node_type: Node type abbreviation (e.g., 'Gene', 'Pathway')
    
    Returns:
        dict: {degree: frequency} normalized to sum to 1
    """
    # Load metagraph to find all edges involving this node type
    metagraph_file = data_dir / 'hetionet-v1.0' / 'hetmat' / 'metagraph.json'
    import json
    with open(metagraph_file, 'r') as f:
        metagraph = json.load(f)
    
    # Find edges where node_type appears
    node_degrees = []
    for edge in metagraph['edges']:
        edge_abbrev = edge['abbreviation']
        source_type = edge['source']
        target_type = edge['target']
        
        # Load edge matrix
        edge_file = data_dir / 'hetionet-v1.0' / 'hetmat' / 'edges' / f'{edge_abbrev}.sparse.npz'
        if not edge_file.exists():
            continue
            
        matrix = sp.load_npz(str(edge_file))
        
        # If source matches, use source degrees
        if source_type == node_type:
            degrees = np.array(matrix.sum(axis=1)).flatten()
            node_degrees.extend(degrees[degrees > 0].tolist())
        
        # If target matches, use target degrees
        if target_type == node_type:
            degrees = np.array(matrix.sum(axis=0)).flatten()
            node_degrees.extend(degrees[degrees > 0].tolist())
    
    # Compute frequency distribution
    unique_degrees, counts = np.unique(node_degrees, return_counts=True)
    total_count = counts.sum()
    
    degree_freq = {int(deg): float(count / total_count) 
                   for deg, count in zip(unique_degrees, counts)}
    
    return degree_freq

# Test
gene_deg_dist = get_intermediate_node_degree_distribution('Gene')
print(f"Gene degree distribution has {len(gene_deg_dist)} unique degrees")
print(f"Total probability: {sum(gene_deg_dist.values()):.4f}")

## Compositional Null Calculator

In [ ]:
def predict_edge_probability(source_deg, target_deg, model_data, model_type='rf'):
    """
    Predict edge probability given source and target degrees.
    
    Args:
        source_deg: Source node degree
        target_deg: Target node degree
        model_data: Dictionary containing model and scaler
        model_type: 'rf' or 'poly_logreg'
    
    Returns:
        float: Predicted edge probability
    """
    model = model_data['model']
    scaler = model_data.get('scaler', None)
    
    # Prepare features
    features = np.array([[source_deg, target_deg]])
    
    if scaler is not None:
        features = scaler.transform(features)
    
    # Predict probability
    if model_type == 'rf':
        prob = model.predict_proba(features)[0, 1]
    else:
        prob = model.predict_proba(features)[0, 1]
    
    return float(prob)

def compute_metapath_null_2edge(source_degrees, target_degrees, 
                                edge1_type, edge2_type,
                                intermediate_degree_freq,
                                model_type='rf'):
    """
    Compute compositional null for 2-edge metapath.
    
    P(source -> target via metapath) = Σ_i P(source -> i) * P(i -> target) * freq(i)
    
    Args:
        source_degrees: Array of source node degrees
        target_degrees: Array of target node degrees
        edge1_type: First edge type abbreviation
        edge2_type: Second edge type abbreviation
        intermediate_degree_freq: Dict of {degree: frequency} for intermediate nodes
        model_type: 'rf' or 'poly_logreg'
    
    Returns:
        numpy array: Predicted null probabilities
    """
    # Load models
    edge1_models = load_null_models(edge1_type, model_type)
    edge2_models = load_null_models(edge2_type, model_type)
    
    null_probs = []
    
    for source_deg, target_deg in zip(source_degrees, target_degrees):
        total_prob = 0.0
        
        # Sum over all intermediate node degrees
        for inter_deg, freq in intermediate_degree_freq.items():
            # P(source → intermediate)
            p1 = predict_edge_probability(source_deg, inter_deg, edge1_models, model_type)
            
            # P(intermediate → target)
            p2 = predict_edge_probability(inter_deg, target_deg, edge2_models, model_type)
            
            # Compositional multiplication weighted by frequency
            total_prob += p1 * p2 * freq
        
        null_probs.append(total_prob)
    
    return np.array(null_probs)

print("Compositional null calculator ready")

## Load Hetionet Metapath Frequencies

In [ ]:
def load_hetionet_metapath_data(metapath):
    """
    Load metapath data from Hetionet.
    
    Args:
        metapath: Metapath abbreviation (e.g., 'CbGpPW')
    
    Returns:
        pd.DataFrame: Metapath data with source_degree, target_degree, path_count
    """
    metapath_file = results_dir.parent / 'degree_aware_compositionality' / f'{metapath}_hetionet_paths.csv'
    
    if not metapath_file.exists():
        # Try loading from edge frequency file
        print(f"Metapath file not found: {metapath_file}")
        print(f"Attempting to compute from scratch...")
        return None
    
    df = pd.read_csv(metapath_file)
    return df

def compute_metapath_from_edges(edge1_type, edge2_type, intermediate_node_type):
    """
    Compute metapath counts directly from edge matrices.
    
    Returns:
        pd.DataFrame: With columns source_id, target_id, source_degree, target_degree, path_count
    """
    # Load edge matrices
    edge1_file = data_dir / 'hetionet-v1.0' / 'hetmat' / 'edges' / f'{edge1_type}.sparse.npz'
    edge2_file = data_dir / 'hetionet-v1.0' / 'hetmat' / 'edges' / f'{edge2_type}.sparse.npz'
    
    edge1_matrix = sp.load_npz(str(edge1_file))
    edge2_matrix = sp.load_npz(str(edge2_file))
    
    # Compute metapath: edge1 @ edge2
    metapath_matrix = edge1_matrix @ edge2_matrix
    
    # Get degrees
    source_degrees = np.array(edge1_matrix.sum(axis=1)).flatten()
    target_degrees = np.array(edge2_matrix.sum(axis=0)).flatten()
    
    # Convert to dataframe
    metapath_coo = metapath_matrix.tocoo()
    
    df = pd.DataFrame({
        'source_id': metapath_coo.row,
        'target_id': metapath_coo.col,
        'path_count': metapath_coo.data,
        'source_degree': source_degrees[metapath_coo.row],
        'target_degree': target_degrees[metapath_coo.col]
    })
    
    return df

print("Metapath loading functions ready")

## Define Metapaths to Analyze

In [ ]:
# Define metapaths with their component edges
metapaths_to_analyze = [
    {
        'name': 'CbGpPW',
        'edge1': 'CbG',
        'edge2': 'GpPW',
        'intermediate_node': 'Gene',
        'description': 'Compound binds Gene participates in Pathway'
    },
    {
        'name': 'CtDaG',
        'edge1': 'CtD',
        'edge2': 'DaG',
        'intermediate_node': 'Disease',
        'description': 'Compound treats Disease associates with Gene'
    },
    {
        'name': 'CbGaD',
        'edge1': 'CbG',
        'edge2': 'GaD',
        'intermediate_node': 'Gene',
        'description': 'Compound binds Gene associates with Disease'
    },
    {
        'name': 'CrCbG',
        'edge1': 'CrC',
        'edge2': 'CbG',
        'intermediate_node': 'Compound',
        'description': 'Compound resembles Compound binds Gene'
    },
    {
        'name': 'GuGiG',
        'edge1': 'GuG',
        'edge2': 'GiG',
        'intermediate_node': 'Gene',
        'description': 'Gene upregulates Gene interacts with Gene'
    }
]

print(f"Will analyze {len(metapaths_to_analyze)} metapaths")
for mp in metapaths_to_analyze:
    print(f"  {mp['name']}: {mp['description']}")

## Process Each Metapath

In [ ]:
def analyze_metapath(metapath_info, model_type='rf'):
    """
    Analyze a single metapath: compute null, compare to Hetionet, generate visualizations.
    
    Args:
        metapath_info: Dictionary with metapath information
        model_type: 'rf' or 'poly_logreg'
    
    Returns:
        dict: Analysis results
    """
    metapath_name = metapath_info['name']
    edge1 = metapath_info['edge1']
    edge2 = metapath_info['edge2']
    intermediate_node = metapath_info['intermediate_node']
    
    print(f"\n{'='*60}")
    print(f"Analyzing {metapath_name}: {metapath_info['description']}")
    print(f"{'='*60}")
    
    # Load or compute Hetionet metapath data
    print(f"Loading Hetionet metapath data...")
    hetionet_df = compute_metapath_from_edges(edge1, edge2, intermediate_node)
    print(f"  Found {len(hetionet_df):,} source-target pairs with paths")
    
    # Get intermediate node degree distribution
    print(f"Computing {intermediate_node} degree distribution...")
    inter_deg_freq = get_intermediate_node_degree_distribution(intermediate_node)
    print(f"  {len(inter_deg_freq)} unique degrees")
    
    # Compute compositional null predictions
    print(f"Computing compositional null predictions using {model_type}...")
    null_probs = compute_metapath_null_2edge(
        hetionet_df['source_degree'].values,
        hetionet_df['target_degree'].values,
        edge1, edge2,
        inter_deg_freq,
        model_type=model_type
    )
    
    hetionet_df['null_prediction'] = null_probs
    
    # Normalize path counts to probabilities
    total_paths = hetionet_df['path_count'].sum()
    hetionet_df['path_probability'] = hetionet_df['path_count'] / total_paths
    
    # Statistical tests
    print(f"\nStatistical Analysis:")
    
    # Correlation
    corr, corr_pval = pearsonr(hetionet_df['null_prediction'], hetionet_df['path_probability'])
    print(f"  Correlation: {corr:.4f} (p={corr_pval:.2e})")
    
    # RMSE
    rmse = np.sqrt(np.mean((hetionet_df['null_prediction'] - hetionet_df['path_probability'])**2))
    print(f"  RMSE: {rmse:.6f}")
    
    # KS test
    ks_stat, ks_pval = ks_2samp(hetionet_df['null_prediction'], hetionet_df['path_probability'])
    print(f"  KS test: statistic={ks_stat:.4f}, p={ks_pval:.2e}")
    
    # Identify anomalous paths (high residuals)
    hetionet_df['residual'] = hetionet_df['path_probability'] - hetionet_df['null_prediction']
    hetionet_df['abs_residual'] = np.abs(hetionet_df['residual'])
    
    # Top overrepresented paths (positive residuals)
    top_over = hetionet_df.nlargest(10, 'residual')
    print(f"\n  Top 10 overrepresented paths (actual > null):")
    for idx, row in top_over.iterrows():
        print(f"    Source {row['source_id']} -> Target {row['target_id']}: "
              f"actual={row['path_probability']:.6f}, null={row['null_prediction']:.6f}, "
              f"residual={row['residual']:.6f}")
    
    # Top underrepresented paths (negative residuals)
    top_under = hetionet_df.nsmallest(10, 'residual')
    print(f"\n  Top 10 underrepresented paths (actual < null):")
    for idx, row in top_under.iterrows():
        print(f"    Source {row['source_id']} -> Target {row['target_id']}: "
              f"actual={row['path_probability']:.6f}, null={row['null_prediction']:.6f}, "
              f"residual={row['residual']:.6f}")
    
    # Save results
    output_file = results_dir / f'{metapath_name}_{model_type}_analysis.csv'
    hetionet_df.to_csv(output_file, index=False)
    print(f"\nResults saved to: {output_file}")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # 1. Scatter: Null vs Actual
    ax = axes[0, 0]
    ax.scatter(hetionet_df['null_prediction'], hetionet_df['path_probability'], 
               alpha=0.3, s=10)
    ax.plot([0, hetionet_df[['null_prediction', 'path_probability']].max().max()],
            [0, hetionet_df[['null_prediction', 'path_probability']].max().max()],
            'r--', alpha=0.5, label='y=x')
    ax.set_xlabel('Null Prediction')
    ax.set_ylabel('Hetionet Path Probability')
    ax.set_title(f'{metapath_name}: Null vs Actual\nr={corr:.4f}, RMSE={rmse:.6f}')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. Residual distribution
    ax = axes[0, 1]
    ax.hist(hetionet_df['residual'], bins=50, edgecolor='black', alpha=0.7)
    ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Zero')
    ax.set_xlabel('Residual (Actual - Null)')
    ax.set_ylabel('Frequency')
    ax.set_title('Residual Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 3. Residual vs source degree
    ax = axes[1, 0]
    ax.scatter(hetionet_df['source_degree'], hetionet_df['residual'], 
               alpha=0.3, s=10)
    ax.axhline(0, color='red', linestyle='--', linewidth=1)
    ax.set_xlabel('Source Degree')
    ax.set_ylabel('Residual')
    ax.set_title('Residual vs Source Degree')
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)
    
    # 4. Residual vs target degree
    ax = axes[1, 1]
    ax.scatter(hetionet_df['target_degree'], hetionet_df['residual'], 
               alpha=0.3, s=10)
    ax.axhline(0, color='red', linestyle='--', linewidth=1)
    ax.set_xlabel('Target Degree')
    ax.set_ylabel('Residual')
    ax.set_title('Residual vs Target Degree')
    ax.set_xscale('log')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plot_file = results_dir / f'{metapath_name}_{model_type}_analysis.png'
    plt.savefig(plot_file, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Plots saved to: {plot_file}")
    
    # Return summary
    return {
        'metapath': metapath_name,
        'model_type': model_type,
        'n_pairs': len(hetionet_df),
        'correlation': corr,
        'correlation_pval': corr_pval,
        'rmse': rmse,
        'ks_statistic': ks_stat,
        'ks_pval': ks_pval,
        'mean_residual': hetionet_df['residual'].mean(),
        'std_residual': hetionet_df['residual'].std()
    }

print("Analysis function ready")

## Run Analysis for All Metapaths

In [ ]:
# Analyze with Random Forest models
results_rf = []

for metapath_info in metapaths_to_analyze:
    try:
        result = analyze_metapath(metapath_info, model_type='rf')
        results_rf.append(result)
    except Exception as e:
        print(f"ERROR analyzing {metapath_info['name']}: {str(e)}")
        import traceback
        traceback.print_exc()

# Create summary dataframe
summary_rf = pd.DataFrame(results_rf)
print("\n" + "="*60)
print("SUMMARY: Random Forest Models")
print("="*60)
print(summary_rf.to_string(index=False))

summary_rf.to_csv(results_dir / 'summary_rf.csv', index=False)
print(f"\nSummary saved to: {results_dir / 'summary_rf.csv'}")

In [ ]:
# Analyze with Polynomial Logistic Regression models
results_poly = []

for metapath_info in metapaths_to_analyze:
    try:
        result = analyze_metapath(metapath_info, model_type='poly_logreg')
        results_poly.append(result)
    except Exception as e:
        print(f"ERROR analyzing {metapath_info['name']}: {str(e)}")
        import traceback
        traceback.print_exc()

# Create summary dataframe
summary_poly = pd.DataFrame(results_poly)
print("\n" + "="*60)
print("SUMMARY: Polynomial Logistic Regression Models")
print("="*60)
print(summary_poly.to_string(index=False))

summary_poly.to_csv(results_dir / 'summary_poly_logreg.csv', index=False)
print(f"\nSummary saved to: {results_dir / 'summary_poly_logreg.csv'}")

## Compare Model Performance

In [ ]:
# Merge summaries for comparison
comparison = summary_rf.merge(summary_poly, on='metapath', suffixes=('_rf', '_poly'))

# Create comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Correlation comparison
ax = axes[0, 0]
x = np.arange(len(comparison))
width = 0.35
ax.bar(x - width/2, comparison['correlation_rf'], width, label='Random Forest', alpha=0.8)
ax.bar(x + width/2, comparison['correlation_poly'], width, label='Poly LogReg', alpha=0.8)
ax.set_xlabel('Metapath')
ax.set_ylabel('Correlation')
ax.set_title('Model Correlation Comparison')
ax.set_xticks(x)
ax.set_xticklabels(comparison['metapath'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 2. RMSE comparison
ax = axes[0, 1]
ax.bar(x - width/2, comparison['rmse_rf'], width, label='Random Forest', alpha=0.8)
ax.bar(x + width/2, comparison['rmse_poly'], width, label='Poly LogReg', alpha=0.8)
ax.set_xlabel('Metapath')
ax.set_ylabel('RMSE')
ax.set_title('Model RMSE Comparison (lower is better)')
ax.set_xticks(x)
ax.set_xticklabels(comparison['metapath'], rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 3. Scatter: RF vs Poly correlation
ax = axes[1, 0]
ax.scatter(comparison['correlation_rf'], comparison['correlation_poly'], s=100)
for idx, row in comparison.iterrows():
    ax.annotate(row['metapath'], (row['correlation_rf'], row['correlation_poly']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)
lims = [min(comparison['correlation_rf'].min(), comparison['correlation_poly'].min()) - 0.05,
        max(comparison['correlation_rf'].max(), comparison['correlation_poly'].max()) + 0.05]
ax.plot(lims, lims, 'r--', alpha=0.5, label='y=x')
ax.set_xlabel('Random Forest Correlation')
ax.set_ylabel('Poly LogReg Correlation')
ax.set_title('Correlation: RF vs Poly LogReg')
ax.legend()
ax.grid(True, alpha=0.3)

# 4. Scatter: RF vs Poly RMSE
ax = axes[1, 1]
ax.scatter(comparison['rmse_rf'], comparison['rmse_poly'], s=100)
for idx, row in comparison.iterrows():
    ax.annotate(row['metapath'], (row['rmse_rf'], row['rmse_poly']),
                xytext=(5, 5), textcoords='offset points', fontsize=8)
lims = [min(comparison['rmse_rf'].min(), comparison['rmse_poly'].min()) - 0.01,
        max(comparison['rmse_rf'].max(), comparison['rmse_poly'].max()) + 0.01]
ax.plot(lims, lims, 'r--', alpha=0.5, label='y=x')
ax.set_xlabel('Random Forest RMSE')
ax.set_ylabel('Poly LogReg RMSE')
ax.set_title('RMSE: RF vs Poly LogReg')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
comparison_plot_file = results_dir / 'model_comparison.png'
plt.savefig(comparison_plot_file, dpi=300, bbox_inches='tight')
plt.close()

print(f"\nModel comparison plot saved to: {comparison_plot_file}")

# Print winner for each metapath
print("\n" + "="*60)
print("BEST MODEL BY METAPATH (by correlation)")
print("="*60)
for idx, row in comparison.iterrows():
    if row['correlation_rf'] > row['correlation_poly']:
        winner = 'Random Forest'
        winner_corr = row['correlation_rf']
        winner_rmse = row['rmse_rf']
    else:
        winner = 'Poly LogReg'
        winner_corr = row['correlation_poly']
        winner_rmse = row['rmse_poly']
    print(f"{row['metapath']:15s} -> {winner:20s} (r={winner_corr:.4f}, RMSE={winner_rmse:.6f})")

## Summary and Conclusions

In [ ]:
print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)

print(f"\nAnalyzed {len(metapaths_to_analyze)} metapaths with 2 model types")
print(f"Results directory: {results_dir}")

print("\nOutput files:")
for f in sorted(results_dir.glob('*')):
    print(f"  {f.name}")

print("\nOverall Statistics:")
print(f"  RF - Mean correlation: {summary_rf['correlation'].mean():.4f} ± {summary_rf['correlation'].std():.4f}")
print(f"  RF - Mean RMSE: {summary_rf['rmse'].mean():.6f} ± {summary_rf['rmse'].std():.6f}")
print(f"  Poly - Mean correlation: {summary_poly['correlation'].mean():.4f} ± {summary_poly['correlation'].std():.4f}")
print(f"  Poly - Mean RMSE: {summary_poly['rmse'].mean():.6f} ± {summary_poly['rmse'].std():.6f}")

print("\nNext steps:")
print("  1. Review top over/underrepresented paths for biological insights")
print("  2. Extend to longer metapaths (3+ edges) using compositional approach")
print("  3. Integrate with downstream analyses (e.g., drug repurposing)")